In [1]:
import sys
sys.path.append('../scared_lbc')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import cupy as _cu
from tqdm.auto import tqdm
import numpy as np
import math

In [4]:
import scared_cu

In [5]:
ABS        = 1
ID         = 2
ID_INV     = 3
OPF        = 7
ABS_ALT    = 8
OPF_ALT    = 9

In [6]:
reduction = scared_cu.lbc.modop.Reduction_Q2Q2(3329, 'int32')

model = scared_cu.models.AbsoluteValue()


N = 2000
sigma = 0
conv = 500

n = 100 #don't touch
n_step = 50

neg_trick = False

prng_off = False

incomplete = 0

In [7]:
def hocpa_wrapper(ths, reduction, model, step=1000, conv_step=1000, neg_trick=False, subkeys=None, early_stop=True, pp=True):

    if incomplete == 0:
        selection_function = scared_cu.lbc.selection_functions.BaseMul(reduction=reduction, words=subkeys)
    else:
        selection_function = scared_cu.lbc.selection_functions.BaseMulIncomplete(reduction=reduction, words=subkeys)
    cpa = scared_cu.CPAAttackAlternative(selection_function=selection_function,
                                               model=model,
                                               discriminant=scared_cu.maxabs,
                                               convergence_step=conv_step)


    mean = ths.samples.mean(axis=0)
    _n = ths.samples.shape[1] // 2
    pp_ = scared_cu.preprocesses.high_order.CenteredProduct(mean=mean, frame_1=range(0,_n), frame_2=range(_n,2*_n), mode='same')

    for i in tqdm(range(0, len(ths), step)):
        if pp:
            container = scared_cu.Container(ths[i:i + step], preprocesses=[pp_])
        else:
            container = scared_cu.Container(ths[i:i + step])
        cpa.run(container)

        if early_stop:
            succs = util.succ_ratio(ths.s[0, subkeys], reduction.q, conv, cpa)
            if succs[-1] == 1.0:
                break

    return cpa

In [ ]:
ths = scared_cu.lbc.simulated_traces.collect_traces_basemult(N=N, n=n, sigma=sigma, reduction=reduction, incomplete=incomplete,
                              const_seed=True, two_step=False, seed=0, prng_off=prng_off, leak_reduction=True)

print(ths)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [9]:
succs = None
for n_i in tqdm(range(0, n, n_step)):
    subkeys = _cu.arange(n_i, n_i + n_step)
    assert ((ths.s % reduction.q) == ((ths.s0 + ths.s1) % reduction.q)).all()
    cpa = hocpa_wrapper(ths[:N], reduction, model, step=conv, conv_step=conv, neg_trick=neg_trick, subkeys=subkeys, early_stop=False, pp=not prng_off)
    succs_i_ = scared_cu._utils.succ_ratio(ths.s[0, subkeys], reduction.q, conv, cpa, incomplete=incomplete!=0)
    succs_i = _cu.zeros(N//conv)
    for i in range(len(succs_i_)):
        succs_i[i] = succs_i_[i]
    for i in range(len(succs_i_), len(succs_i)):
        succs_i[i] = succs_i_[-1]
    if succs is None:
        succs = _cu.array(succs_i)
    else:
        succs += _cu.array(succs_i)
    
    for x,y in zip(range(conv,N+1,conv),succs):
        print(f'({math.log(x)/math.log(10)},{y})',end='')
    print()
    for i in range(n_i, n_i + n_step):
        print(f'{ths.s[0, i]}-{cpa.scores[:,i-n_i].argsort()[-2]}', end='\t')
    print()

    if n_i != (n - n_step):
        del cpa


succs /= (n//n_step)
# assert (mmia.data == (-1)).sum() == 0


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

(500, 3329, 50)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
(500, 3329, 50)
(500, 3329, 50)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
(500, 3329, 50)
(500, 3329, 50)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
(500, 3329, 50)
(500, 3329, 50)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
(500, 3329, 50)
(2.6989700043360183,1.0)(2.9999999999999996,1.0)(3.176091259055681,1.0)(3.301029995663981,1.0)
-586-2743	696-2633	298-3031	-3-3326	-466-2863	1340-1989	449-2880	-1334-1995	-15-3314	-1019-2310	-721-2608	989-2340	-791-2538	1266-2063	938-2391	1121-2208	-293-30

  0%|          | 0/4 [00:00<?, ?it/s]

(500, 3329, 50)
[50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]
(500, 3329, 50)
(500, 3329, 50)
[50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]
(500, 3329, 50)
(500, 3329, 50)
[50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]
(500, 3329, 50)
(500, 3329, 50)
[50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]
(500, 3329, 50)
(2.6989700043360183,2.0)(2.9999999999999996,2.0)(3.176091259055681,2.0)(3.301029995663981,2.0)
294-3035	-594-2735	-902-2427	-80-3249	-47-3282	-272-3057	-1584-1745	-521-2808	1137-2192	-566-2763	221-3108	-1632-1697	-968-2361	-128-3201	-1155-2174	1628-1701	-3

In [10]:
for x,y in zip(range(conv,N+1,conv),succs):
    print(f'({math.log(x)/math.log(10)},{y})',end='')

(2.6989700043360183,1.0)(2.9999999999999996,1.0)(3.176091259055681,1.0)(3.301029995663981,1.0)

In [11]:
np.array([1,2,3])[[3,4,5]]

IndexError: index 3 is out of bounds for axis 0 with size 3